In [1]:
import os
sep_local = os.path.sep
sep_local

'/'

In [2]:
import sys
sys.path.append('..'+sep_local+'..')

In [3]:
#done in case of linux
#import os
#os.chdir("/home/azeghost/git/Generative_Models")

In [4]:
import tensorflow as tf

In [5]:
tf.__version__

'2.1.0'

In [6]:
dataset_name = 'mnist'

In [7]:
import tensorflow_datasets as tfds

info = tfds.builder(dataset_name).info

print(info)

tfds.core.DatasetInfo(
    name='mnist',
    version=1.0.0,
    description='The MNIST database of handwritten digits.',
    homepage='http://yann.lecun.com/exdb/mnist/',
    features=FeaturesDict({
        'image': Image(shape=(28, 28, 1), dtype=tf.uint8),
        'label': ClassLabel(shape=(), dtype=tf.int64, num_classes=10),
    }),
    total_num_examples=70000,
    splits={
        'test': 10000,
        'train': 60000,
    },
    supervised_keys=('image', 'label'),
    citation="""@article{lecun2010mnist,
      title={MNIST handwritten digit database},
      author={LeCun, Yann and Cortes, Corinna and Burges, CJ},
      journal={ATT Labs [Online]. Available: http://yann. lecun. com/exdb/mnist},
      volume={2},
      year={2010}
    }""",
    redistribution_info=,
)



In [8]:
intermediate_dim = 20
inputs_shape=(28, 28, 1) # image shape
batch_size = 100
latent_dim = 50

In [9]:
TRAIN_BUF = 600
TEST_BUF = 100


In [10]:
import tensorflow_datasets as tfds
# Construct a tf.data.Dataset
train_ds = tfds.load(name=dataset_name, split=tfds.Split.TRAIN).shuffle(TRAIN_BUF).batch(batch_size)
try:
    test_ds = tfds.load(name=dataset_name, split=tfds.Split.TEST).shuffle(TEST_BUF).batch(batch_size)
except:
    test_ds = tfds.load(name=dataset_name, split=tfds.Split.TRAIN).shuffle(TEST_BUF).batch(batch_size)


In [11]:
_instance_scale=1.0
for data in train_ds:
    _instance_scale = float(data['image'][0].numpy().max())
    break

In [12]:
_instance_scale

254.0

In [13]:
enc_lays2 = [
    tf.keras.layers.Conv2D(filters=32, kernel_size=3, strides=(2, 2), activation='relu'),
    tf.keras.layers.Conv2D(filters=64, kernel_size=3, strides=(2, 2), activation='relu'),
    tf.keras.layers.Flatten(),
    # No activation
    tf.keras.layers.Dense(latent_dim)
]

dec_lays2 = [
    tf.keras.layers.Dense(units=7*7*32, activation=tf.nn.relu),
    tf.keras.layers.Reshape(target_shape=(7, 7, 32)),
    tf.keras.layers.Conv2DTranspose(filters=64, kernel_size=3, strides=(2, 2), padding="SAME", activation='relu'),
    tf.keras.layers.Conv2DTranspose(filters=32, kernel_size=3, strides=(2, 2), padding="SAME", activation='relu'),
    
    # No activation
    tf.keras.layers.Conv2DTranspose(filters=1, kernel_size=3, strides=(1, 1), padding="SAME")
]

In [14]:
enc_mean_lays = [tf.keras.layers.Dense(units=intermediate_dim, activation='relu'),
            tf.keras.layers.Dense(units=intermediate_dim, activation='relu'),
            tf.keras.layers.Dense(units=intermediate_dim, activation='relu')]

enc_var_lays = [tf.keras.layers.Dense(units=intermediate_dim, activation='relu'),
            tf.keras.layers.Dense(units=intermediate_dim, activation='relu'),
            tf.keras.layers.Dense(units=intermediate_dim, activation='relu')]

dec_lays = [tf.keras.layers.Dense(units=2*intermediate_dim, activation='relu'),
            tf.keras.layers.Dense(units=2*intermediate_dim, activation='relu'),
            tf.keras.layers.Dense(units=2*intermediate_dim, activation='relu')]

## First Time

In [ ]:
#from utils.data_and_files.file_utils import make_random_string
#from time import gmtime, strftime

#model_name = 'AE_' + make_random_string(5) + strftime("%a_%d_%b_%Y_%H_%M", gmtime())
#print(model_name)

In [15]:
model_name = dataset_name+'AE'

In [16]:
recoding_dir='..'+sep_local+'..'+sep_local+'recoding'+sep_local+model_name

In [17]:
import os

In [18]:
os.path.join(recoding_dir, model_name)

'../../recoding/mnistAE/mnistAE'

In [20]:
import os
sep_local = os.path.sep
os.chdir('..'+sep_local+'..')

In [21]:
pwd

'/home/azeghost/git/new_GAN_models/Generative_Models'

In [22]:
#from training.traditional.autoencoders.AE import AE
from training.traditional.autoencoders.autoencoder import autoencoder as AE

Using TensorFlow backend.


In [27]:
variables_params = \
[
    {
        'name': 'inference', 
        'inputs_shape':inputs_shape,
        'outputs_shape':latent_dim,
        'layers': enc_mean_lays
    }

    ,
    
        {
        'name': 'generative', 
        'inputs_shape':latent_dim,
        'outputs_shape':inputs_shape,
        'layers':dec_lays
    }
]

In [28]:
from utils.data_and_files.file_utils import create_if_not_exist
_restore = os.path.join(recoding_dir, 'var_save_dir')
create_if_not_exist(_restore)
_restore
ae = AE( 
    name=model_name,
    inputs_shape=inputs_shape,
    outputs_shape=inputs_shape,
    latent_dim=latent_dim,
    variables_params=variables_params, 
    filepath=None
    )

  WARNING  | None
  WARNING  | None


Model: "inference"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_2 (Dense)              (None, 28, 28, 20)        40        
_________________________________________________________________
dense_3 (Dense)              (None, 28, 28, 20)        420       
_________________________________________________________________
dense_4 (Dense)              (None, 28, 28, 20)        420       
_________________________________________________________________
batch_normalization (BatchNo (None, 28, 28, 20)        80        
_________________________________________________________________
dropout (Dropout)            (None, 28, 28, 20)        0         
_________________________________________________________________
activity_regularization (Act (None, 28, 28, 20)        0         
_________________________________________________________________
inference_outputs (Activatio (None, 28, 28, 20)        0 

ValueError: Input 0 of layer generative is incompatible with the layer: expected axis -1 of input shape to have value 50 but received input with shape [None, 28, 28, 20]

In [ ]:
ae.compile(metrics=None)

In [ ]:
from training.callbacks.progress_bar import NotebookPrograssBar
from training.callbacks.sample_generation import SampleGeneration
from training.callbacks.save_model import ModelSaver
progbar = NotebookPrograssBar(leave_outer=False)
es = tf.keras.callbacks.EarlyStopping(
    monitor='loss', 
    min_delta=1e-12, 
    patience=5, 
    verbose=1, 
    restore_best_weights=True
)
ms = ModelSaver(filepath=_restore,save_freq=5)

csv_dir = os.path.join(recoding_dir, 'csv_dir')
create_if_not_exist(csv_dir)
csv_dir = os.path.join(csv_dir, ae.name+'.csv')
csv_log = tf.keras.callbacks.CSVLogger(csv_dir, append=True)
csv_dir
image_gen_dir = os.path.join(recoding_dir, 'image_gen_dir')
create_if_not_exist(image_gen_dir)
sg = SampleGeneration(latent_shape=6, filepath=image_gen_dir, gen_freq=5, save_img=True, gray_plot=True)

In [ ]:
def cast_batch(batch):
    return tf.cast(batch['image'], dtype=tf.float32), tf.cast(batch['image'], dtype=tf.float32)
ds = train_ds.map(cast_batch)

In [ ]:

ae.fit_generator(
    generator=ds,
    steps_per_epoch=50,
    epochs=100, 
    verbose=0,
    callbacks=[progbar, es, ms, csv_log, sg, gt_mertics],
    workers=-1,
    use_multiprocessing=True
)

In [ ]:
import matplotlib.pyplot as plt

In [ ]:
plt.imshow(image.reshape(28,28))

In [ ]:
reconst = ae.decode(ae.encode(image))

In [ ]:
plt.imshow(reconst.numpy().reshape(28,28))